# RDS連線

# 架構圖

In [7]:
from diagrams import Diagram, Cluster, Node, Edge
from diagrams.aws.database import RDS
from diagrams.aws.migration import DMS
from diagrams.aws.analytics import KinesisDataStreams, Glue, Kinesis, KinesisDataFirehose
from diagrams.aws.storage import S3
from diagrams.aws.compute import Lambda, ECS, EKS, ElasticBeanstalk
from diagrams.aws.integration import SNS, SQS, Eventbridge, StepFunctions
from diagrams.aws.network import APIGateway, VPC
from diagrams.aws.ml import *
from diagrams.onprem.client import User, Users
from diagrams.generic.device import Mobile,Tablet
# 設定字體大小 (fontsize)
graph_attr = {
    "fontsize": "18"  # 整體圖的字體大小
}

node_attr = {
    "fontsize": "14"  # 節點的字體大小
}

edge_attr = {
    "fontsize": "14"  # 邊線的字體大小 (如果有邊線標籤)
}
with Diagram("",
              show=False, 
              filename="data_architecture", 
              direction="LR", 
              graph_attr=graph_attr, 
              node_attr=node_attr, 
              edge_attr=edge_attr):
    with Cluster("CDC"):
        # RDS 資料來源
        rds = RDS("PostgreSQL")
        # DMS 監控 RDS
        dms = DMS("DMS")

    # Kinesis & Lambda 
    with Cluster("Streaming Processing"):
        kinesis_stream = KinesisDataStreams("Kinesis Streaming")
        kinesis_firehose = KinesisDataFirehose("Firehose Streaming")
        lambda_func = Lambda("Lambda")
    # API Gateway 與 SNS
    api_gateway = APIGateway("API Gateway")
    rds >> kinesis_stream >> kinesis_firehose
    kinesis_firehose >> lambda_func >> api_gateway